In [1]:
from bs4 import BeautifulSoup
import httpx
from typing import Callable
from functools import reduce

PTT_OVER_18_HEADER = {"cookie": "over18=1"}

In [2]:
 # static function
 
 # add some new function 
    
def page_to_simple_dict(html_str: str , func_list:list[Callable] = None) -> dict:
    """
    format like
    '作者': 'ReiKuromiya (ReiKuromiya)',
    '標題': '[正妹] 周子瑜',
    '時間': 'Sun Jan  1 00:26:06 2023',
    'Year': '2023',
    'Month': 'Jan',
    'Day': '',
    'Week': 'Sun',
    'Time': '00:26:06',
    # 'Body': [(..., ...),  ...]}
    """

    def to_detail_date(date_str: str) -> dict:
        detail_date = date_str.split(" ")

        if "" in detail_date:
            detail_date.remove("")
        return {
            "Year": detail_date[-1],
            "Month": detail_date[1],
            "Day": detail_date[2],
            "Week": detail_date[0],
            "Time": detail_date[-2],
        }

    # header = {"cookie": "over18=1"}
    # result = httpx.get(url=url, headers=header)

    soup = BeautifulSoup(html_str, "html.parser")

    # get main data
    body_data = soup.find("div", class_="bbs-screen bbs-content", id="main-content")

    # get header data
    header_data = body_data.find_all("div", class_="article-metaline")

    tab_list = [
        str(line.find("span", class_="article-meta-tag").string)
        for line in header_data
    ]

    value_list = [
        str(line.find("span", class_="article-meta-value").string)
        for line in header_data
    ]

    header_dict = dict(zip(tab_list, value_list))

    if "時間" not in header_dict:
        # get text body #https://www.ptt.cc/bbs/Beauty/M.1690589266.A.166.html
        process_text = body_data.contents[2]
        process_text = process_text.split("\n")[:2]
        process_text = [item.split(":") for item in process_text]
        process_result_dict = {item[0]: item[1] for item in process_text}
        header_dict |= process_result_dict

    # image src
    images_catch_link = body_data.find_all("div", class_="richcontent")

    images_catch_link = [
        item.get("src")
        for image in images_catch_link
        if (item := image.find("img"))
    ]

    # image link lists
    image_link = body_data.find_all("a")
    image_link = [
        link_str
        for link in image_link
        if any(
            substring in (link_str := str(link.string))
            for substring in [".png", ".jpg", "jpeg", ".gif"]
        )
    ]

    page_data = (
        header_dict
        | to_detail_date(header_dict["時間"])
        | {
            "image_catch_list": images_catch_link,
            "image_link": image_link,
        }
    )
    
    if func_list is not None:
        addition_dict_list = reduce(lambda a , b : a | b , [func(soup) for func in func_list] )
        page_data |= addition_dict_list
        

    return page_data

In [3]:
html_page = httpx.get("https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5B5.html" , headers=PTT_OVER_18_HEADER)
# html_page = httpx.get("https://www.ptt.cc/bbs/Beauty/M.1672503968.A.5B5.html" , headers=PTT_OVER_18_HEADER)

In [32]:
def get_like_boo_count_dict(soup: BeautifulSoup) -> dict:
    
    def push_list_to_dict(item:BeautifulSoup) -> dict:
        
        
        span_item = [thing.string for thing in  item.find_all("span")]
        span_item = [str(thing).strip() for thing in  span_item]
        
        ip ,date_str , time_str = span_item[3].split(" ")
        
        return {
            "like_boo_type": span_item[0],
            "user_name": span_item[1],
            "comment" : span_item[2].replace(":" ,"").strip(),
            "ip" : ip,
            "date" : date_str,
            "time" : time_str,
        }
    
    push_list = soup.find_all("div" , class_="push")
    
    
    test_item  = [push_list_to_dict(item=item) for item in push_list]
    print(test_item)
    
    like_list = list(filter(lambda x : x["like_boo_type"] =="推" , test_item))    
    boo_list = list(filter(lambda x : x["like_boo_type"] =="噓" , test_item))    
    mid_list = list(filter(lambda x : x["like_boo_type"] =="→" , test_item))    
    return {"like" : like_list , "boo" : boo_list , "mid" : mid_list}

In [35]:
page_dict = page_to_simple_dict(html_page.text , [get_like_boo_count_dict])
page_dict 

[{'like_boo_type': '推', 'user_name': 'picachuln', 'comment': '推', 'ip': '49.215.33.17', 'date': '01/01', 'time': '00:27'}, {'like_boo_type': '推', 'user_name': 'qd6590', 'comment': '超正推 樓下說很普', 'ip': '49.216.101.82', 'date': '01/01', 'time': '00:28'}, {'like_boo_type': '推', 'user_name': 'Dogman7788', 'comment': '長大了', 'ip': '111.248.97.227', 'date': '01/01', 'time': '00:29'}, {'like_boo_type': '推', 'user_name': 'michael0419', 'comment': '難得露溝', 'ip': '49.216.53.99', 'date': '01/01', 'time': '00:29'}, {'like_boo_type': '推', 'user_name': 'phrita', 'comment': '長大了', 'ip': '122.121.226.81', 'date': '01/01', 'time': '00:29'}, {'like_boo_type': '推', 'user_name': 'lindada78', 'comment': '推', 'ip': '223.136.62.37', 'date': '01/01', 'time': '00:33'}, {'like_boo_type': '推', 'user_name': 'Krishna', 'comment': '噫~~蓋高尚', 'ip': '123.193.251.73', 'date': '01/01', 'time': '00:33'}, {'like_boo_type': '推', 'user_name': 'Ashen19', 'comment': '子子推推', 'ip': '27.247.133.110', 'date': '01/01', 'time': '00:37'

{'作者': 'ReiKuromiya (ReiKuromiya)',
 '標題': '[正妹] 周子瑜',
 '時間': 'Sun Jan  1 00:26:06 2023',
 'Year': '2023',
 'Month': 'Jan',
 'Day': '1',
 'Week': 'Sun',
 'Time': '00:26:06',
 'image_catch_list': ['https://cache.ptt.cc/c/https/i.imgur.com/BdmZ7Psl.jpg?e=1709288787&s=Oi9afBeydG3u17FZ886nqQ',
  'https://cache.ptt.cc/c/https/i.imgur.com/bBiw4ISl.jpg?e=1709257568&s=fTMSE-0hbwRoJSgozpe5ZA',
  'https://cache.ptt.cc/c/https/i.imgur.com/KDmRgegl.jpg?e=1709261149&s=1PMEkT1tGxY4FVgaofjdzg',
  'https://cache.ptt.cc/c/https/i.imgur.com/hRHpy8Xl.jpg?e=1709256477&s=Ym5hezDGtSl3g1_qwmZasw',
  'https://cache.ptt.cc/c/https/i.imgur.com/P5rI8UMl.jpg?e=1709253540&s=DcFO2D6xOnxhVa-xHtKkGg',
  'https://cache.ptt.cc/c/https/i.imgur.com/Uq0oG4El.jpg?e=1709271739&s=VsgZTRQdGxr6PZekOkbfEA',
  'https://cache.ptt.cc/c/https/i.imgur.com/pPXBHuZl.jpg?e=1709303665&s=8VM2J0Pu6VIvSxIb-VY-jw',
  'https://cache.ptt.cc/c/https/i.imgur.com/dFiDDhl.jpg?e=1709302537&s=3eI8qWuJJIkEaGFPnzl83w',
  'https://cache.ptt.cc/c/https